## **Imports**

In [284]:
import pandas as pd
import numpy as np
import importlib
from application.modeling import disbursements
from application.modeling import interest_income
from application.modeling import other_income
from application.modeling import helper
from application.modeling import depreciation
from application.modeling import expenses
from application.modeling import income_statement
from application.modeling import direct_cashflow
pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:,.2f}".format

## **Inputs**

In [285]:
VALUATION_DATE = "2023-01"
MONTHS_TO_FORECAST = 12
IMTT = 0.01

In [286]:
parameters = pd.read_csv("./data_templates/parameters.csv", index_col=0)
parameters.columns = pd.period_range(VALUATION_DATE,periods=int(parameters.columns[-1]), freq="M")
parameters

,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
AGENT_CONTRIBUTION_PERCENT,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10
BUSINESS_ACQUISITION_PERCENT,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
INFLATION_RATE,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
MONTHS_TO_FORECAST,12.00,12.00,12.00,12.00,12.00,12.00,12.00,12.00,12.00,12.00,12.00,12.00
PENSION_AND_STATUROTY_CONTRIBUTIONS_PERCENT,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11
TAX_RATE,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25
AGENT_COMMISSION,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
AGENT_SALARY,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00
CREDIT_OFFICER_COMMISSION,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
CREDIT_OFFICER_SALARY,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00


In [287]:
details_of_existing_assets = pd.read_csv("./data_templates/details_of_existing_assets.csv")
details_of_existing_assets

,asset,depreciation,net_book_value,remaining_useful_life,cost,useful_life_when_purchased,method
0,Vehicles,0.20,10000,5,20000,10,straight_line
1,Buildings,0.10,50000,20,80000,25,reducing_balance


In [288]:
details_of_new_assets = pd.read_csv("./data_templates/details_of_new_assets.csv")
details_of_new_assets

,asset,depreciation,net_book_value,remaining_useful_life,method,purchase_date,cost
0,Vehicle,0.25,10000,5,reducing_balance,01/06/2023,10000
1,Building,0.10,50000,20,reducing_balance,01/09/2023,50000


In [289]:
details_of_existing_long_term_borrowing = pd.read_csv("./data_templates/details_of_existing_long_term_borrowing.csv")
details_of_existing_long_term_borrowing

,company,principal,interest_rate,remaining_loan_term,loan_term,method
0,Zimnat Lion Insurance,92800,0.06,60,60,straight_line
1,Zimnat Lion Insurance,50000,0.06,12,12,straight_line
2,Grandre,160000,0.06,36,36,straight_line
3,Minerva,15000,0.08,36,36,straight_line
4,Masawara,5613,0.06,60,60,straight_line
5,Zam-Cargo,364000,0.05,36,36,straight_line


In [290]:
details_of_new_long_term_borrowing = pd.read_csv("./data_templates/details_of_new_long_term_borrowing.csv")
details_of_new_long_term_borrowing

,company,principal,interest_rate,loan_term,remaining_loan_term,loan_start_date,method
0,ZMF,200000,0.34,36,36,01/08/2023,straight_line
1,ZAM,400000,0.25,60,60,01/09/2023,straight_line
2,NBS,500000,0.18,24,24,01/10/2023,straight_line
3,ZB,500000,0.21,18,18,01/10/2023,straight_line


In [291]:
details_of_new_short_term_borrowing = pd.read_csv("./data_templates/details_of_new_short_term_borrowing.csv")
details_of_new_short_term_borrowing

,company,principal,interest_rate,loan_term,remaining_loan_term,loan_start_date,method
0,ZMF,200000,0.34,12,12,01/07/2023,straight_line
1,ZAM,400000,0.25,12,12,01/07/2023,reducing_balance
2,NBS,500000,0.18,12,12,01/09/2023,straight_line
3,ZB,500000,0.21,12,12,01/10/2023,straight_line


In [292]:
details_of_existing_short_term_borrowing = pd.read_csv("./data_templates/details_of_existing_short_term_borrowing.csv")
details_of_existing_short_term_borrowing

,company,principal,interest_rate,remaining_loan_term,loan_term,method
0,Zimnat Life,50000,0.14,10,12,straight_line
1,Zimnat Life,30000,0.13,12,12,straight_line
2,Grandre,100000,0.12,7,12,straight_line
3,FCB,120383,0.18,5,12,straight_line
4,Nurture,500000,0.25,8,12,straight_line
5,NMB,450000,0.22,11,12,straight_line
6,Beramasamba,30000,0.13,3,12,straight_line
7,Dandemutande,25000,0.12,7,12,reducing_balance


In [293]:
static_inputs_income_statement = pd.read_csv("./data_templates/static_inputs_income_statement.csv", index_col=0)
static_inputs_income_statement

,Jan-23,Feb-23,Mar-23,Apr-23,May-23,Jun-23,Jul-23,Aug-23,Sep-23,Oct-23,Nov-23,Dec-23
CILL,13152,11585,13943,17555,13073,11021,13843,17989,19692,16873,15675,10161
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878
Retrenchments,19268,14887,14859,16331,18571,18684,17208,15276,12062,10064,18006,12568
Staff Welfare,15463,12027,12695,19687,15258,15618,16736,10391,15892,13561,16184,13099
Donations,16278,18392,13104,17215,12454,18996,12731,18154,19762,15056,18110,13840
Legal Fees,0,0,0,0,0,0,0,0,0,0,0,0
Strategic Expenses,0,0,0,0,0,0,0,0,0,0,0,0
Rental Income,0,0,0,0,0,0,0,0,0,0,0,0
Interest Received,0,0,0,0,0,0,0,0,0,0,0,0
Dividends Received,0,0,0,0,0,0,0,0,0,0,0,0


In [294]:
variable_inputs_income_statement = pd.read_csv("./data_templates/variable_inputs_income_statement.csv", index_col=0)
variable_inputs_income_statement

,Aug-22,Sep-22,Oct-22,Nov-22,Dec-22,Jan-23
Training,6028,5502,9493,5870,9488,5206
Travel Costs,6484,5863,7790,5563,9191,6757
Entertainment,6678,8242,9636,6059,6722,8314
Marketing Costs,8157,7625,7729,6597,8060,7693
Group Marketing Costs,9891,8627,6363,6981,6663,6529
Rental Costs,7038,8302,7237,6306,9029,7675
Subscriptions,6282,5709,9744,9413,8748,9779
Insurance,5663,6998,8445,8743,6495,8304
Repairs And Maintenance,8763,6853,9659,6291,8581,8457
Utilities,6636,8696,7999,8152,5698,7160


In [295]:
opening_balances = pd.read_csv("./data_templates/opening_balances.csv")
opening_balances

,TRADE_RECEIVABLES,TRADE_PAYABLES,LOAN_BOOK,CASH_ON_HAND,ISSUED_SHARE_CAPITAL,SHARE_PREMIUM,OTHER_COMPONENTS_OF_EQUITY,TREASURY_SHARES,RETAINED_EARNINGS,LONG_TERM_LOANS,SHORT_TERM_LOANS,INTERCOMPANY_LOANS,DEFERED_TAXATION,OTHER_RECEIVABLES,OTHER_PAYABLES,PROVISION_FOR_TAXATION,PROVISION_FOR_CREDIT_LOSS
0,1000000,940000,2800000,331000,7000,110,0,0,200000,200000,50000,0,0,0,0,0,10000


In [296]:
existing_loans = pd.read_csv("./data_templates/existing_loans.csv", index_col=0)
existing_loans

,disbursement_date,maturity_date,loan_amount,frequency,loan_term_months,remaining_term_months,interest_rate,repayment_amount,outstanding_balance,admin_fee_percentage,credit_insurance_fee_percentage
loan_number,,,,,,,,,,,
40b0775c,43536,46386,2400,12,94,48,0.00,31.39,"1,351.78",0.06,0.05
f5c36555,43873,46386,4600,12,83,48,0.01,92.11,"3,246.86",0.06,0.07
75656bce,44068,46386,2400,12,77,48,0.01,47.81,"1,739.68",0.08,0.05
a7222f00,44611,46021,3800,12,47,36,0.00,83.95,"2,935.72",0.09,0.08
ad00cbf5,43747,46751,5000,12,99,60,0.00,52.09,"3,066.98",0.07,0.05
321ec75b,44137,46751,1700,12,86,60,0.00,24.25,"1,258.85",0.08,0.10
6747e1d4,43606,46021,1600,12,80,36,0.01,31.43,911.36,0.05,0.10
5131c13c,44591,46386,1000,12,60,48,0.01,20.90,835.18,0.05,0.08
a7222f00_1,43520,46021,1200,12,83,36,0.01,18.50,595.83,0.06,0.05


## **New Disbursements**

In [297]:
new_disbursements_df  = disbursements.calculate_new_disbursements(parameters=parameters)
new_disbursements_df

,b2b_disbursements,sme_disbursements,consumer_ssb_disbursements,consumer_pvt_disbursements,total
2023-01,"10,000.00","360,000.00","48,000.00","60,000.00","490,000.00"
2023-02,"10,000.00","432,000.00","60,000.00","75,000.00","592,000.00"
2023-03,"10,000.00","480,000.00","72,000.00","108,000.00","706,000.00"
2023-04,"10,000.00","360,000.00","80,000.00","120,000.00","610,000.00"
2023-05,"10,000.00","324,000.00","96,000.00","144,000.00","622,000.00"
2023-06,"10,000.00","378,000.00","96,000.00","162,000.00","712,000.00"
2023-07,"10,000.00","420,000.00","112,000.00","189,000.00","808,000.00"
2023-08,"10,000.00","595,000.00","120,000.00","202,500.00","1,010,000.00"
2023-09,"10,000.00","700,000.00","120,000.00","225,000.00","1,160,000.00"
2023-10,"10,000.00","990,000.00","120,000.00","225,000.00","1,450,000.00"


### **Calculate Repayment Amount For New Disbursements**

In [298]:
monthly_repayment_new_disbursements_df = interest_income.calculate_monthly_repayments_new_disbursements(new_disbursements_df=new_disbursements_df, parameters=parameters)
monthly_repayment_new_disbursements_df

,sme_monthly_repayment,b2b_monthly_repayment,consumer_ssb_monthly_repayment,consumer_pvt_monthly_repayment
2023-01,"81,451.18","1,627.45","8,006.34","10,007.93"
2023-02,"97,741.42","1,627.45","10,007.93","12,509.91"
2023-03,"108,601.58","1,627.45","12,009.51","18,014.27"
2023-04,"81,451.18","1,627.45","11,172.05","16,758.08"
2023-05,"73,306.06","1,627.45","13,406.46","20,109.69"
2023-06,"85,523.74","1,627.45","13,406.46","22,623.41"
2023-07,"95,026.38","1,627.45","15,640.87","26,393.97"
2023-08,"134,620.70","1,627.45","14,887.07","28,279.26"
2023-09,"158,377.30","1,627.45","14,887.07","31,421.40"
2023-10,"223,990.75","1,627.45","13,705.47","31,421.40"


### **Generate Loan Schedules For New Disbursements**

In [299]:
loan_schedules_for_all_new_disbursements = (
    interest_income.generate_loan_schedules_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df,
        parameters=parameters,
        monthly_repayment_new_disbursements_df=monthly_repayment_new_disbursements_df,
        months_to_forecast=MONTHS_TO_FORECAST,
    )
)
loan_schedules_for_all_new_disbursements.keys()

dict_keys(['sme_loan_schedules', 'b2b_loan_schedules', 'consumer_ssb_loan_schedules', 'consumer_pvt_loan_schedules'])

### **Capital Repayment New Disbursement**

In [300]:
capital_repayment_new_disbursements_df = interest_income.generate_capital_repayment_new_disbursements_df(loan_schedules_for_all_new_disbursements=loan_schedules_for_all_new_disbursements)
capital_repayment_new_disbursements_df

,sme_capital_repayments,b2b_capital_repayments,consumer_pvt_capital_repayments,consumer_ssb_capital_repayments,total
Feb-2023,"47,251.18",627.45,"4,607.93","3,686.34","56,172.91"
Mar-2023,"108,441.46","1,317.65","10,782.55","8,626.04","129,167.71"
Apr-2023,"181,744.98","2,076.87","20,047.25","14,931.90","218,801.00"
May-2023,"246,261.93","2,912.01","27,809.58","20,247.82","297,231.35"
Jun-2023,"312,182.88","3,830.67","37,462.14","26,836.59","380,312.28"
Jul-2023,"391,453.99","4,841.19","48,877.14","34,018.35","479,190.67"
Aug-2023,"402,317.32","5,952.76","62,660.06","42,640.87","513,571.01"
Sep-2023,"420,891.75","7,175.49","78,353.72","50,565.62","556,986.58"
Oct-2023,"444,152.19","8,520.50","96,576.95","59,203.59","608,453.22"
Nov-2023,"534,836.21","10,000.00","106,432.35","59,431.04","710,699.61"


### **Interest Income New Disbursements**


In [301]:
interest_income_new_disbursements_df = interest_income.generate_interest_income_new_disbursements_df(
    loan_schedules_for_all_new_disbursements=loan_schedules_for_all_new_disbursements
)
interest_income_new_disbursements_df

,sme_interest_income,b2b_interest_income,consumer_pvt_interest_income,consumer_ssb_interest_income,total
Feb-2023,"34,200.00","1,000.00","5,400.00","4,320.00","44,920.00"
Mar-2023,"70,751.14","1,937.25","11,735.29","9,388.23","93,811.91"
Apr-2023,"106,049.20","2,805.49","20,484.86","15,091.89","144,431.43"
May-2023,"122,983.43","3,597.80","29,480.60","20,948.01","177,009.85"
Jun-2023,"130,368.54","4,306.60","39,937.74","27,765.71","202,378.60"
Jul-2023,"136,621.17","4,923.53","51,146.15","33,990.42","226,681.27"
Aug-2023,"139,333.04","5,439.41","63,757.21","41,008.77","249,538.43"
Sep-2023,"157,637.89","5,844.14","76,342.80","47,971.09","287,795.92"
Oct-2023,"184,153.18","6,126.59","89,540.97","54,220.18","334,040.92"
Nov-2023,"236,008.72","6,274.54","101,099.04","59,691.86","403,074.16"


### **Calculate Admin Fee On New Disbursements**

In [302]:
admin_fee_for_all_new_disbursements_df = other_income.calculate_admin_fee_for_all_new_disbursements(
    new_disbursements_df=new_disbursements_df,
    parameters=parameters,
    months_to_forecast=MONTHS_TO_FORECAST,
)
admin_fee_for_all_new_disbursements_df

,sme_admin_fee,b2b_admin_fee,consumer_ssb_admin_fee,consumer_pvt_admin_fee,total
Jan-2023,"3,000.00",10.00,184.62,272.73,"3,467.34"
Feb-2023,"6,600.00",20.00,415.38,613.64,"7,649.02"
Mar-2023,"10,600.00",30.00,692.31,"1,104.55","12,426.85"
Apr-2023,"13,600.00",40.00,"1,000.00","1,650.00","16,290.00"
May-2023,"16,300.00",50.00,"1,369.23","2,304.55","20,023.78"
Jun-2023,"19,450.00",60.00,"1,738.46","3,040.91","24,289.37"
Jul-2023,"19,950.00",70.00,"2,169.23","3,900.00","26,089.23"
Aug-2023,"21,308.33",80.00,"2,630.77","4,820.45","28,839.56"
Sep-2023,"23,141.67",90.00,"3,092.31","5,843.18","32,167.16"
Oct-2023,"28,391.67",100.00,"3,553.85","6,865.91","38,911.42"


### **Calculate Credit Life Insurance On New Disbursement**

In [303]:
credit_insurance_fee_for_all_new_disbursements = (
    other_income.calculate_credit_insurance_fee_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df,
        parameters=parameters,
        months_to_forecast=MONTHS_TO_FORECAST,
    )
)

credit_insurance_fee_for_all_new_disbursements

,sme_credit_insurance_fee,b2b_credit_insurance_fee,consumer_ssb_credit_insurance_fee,consumer_pvt_credit_insurance_fee,total
Jan-2023,"1,200.00",0.00,73.85,109.09,"1,382.94"
Feb-2023,"2,640.00",0.00,166.15,245.45,"3,051.61"
Mar-2023,"4,240.00",0.00,276.92,441.82,"4,958.74"
Apr-2023,"5,440.00",0.00,400.00,660.00,"6,500.00"
May-2023,"6,520.00",0.00,547.69,921.82,"7,989.51"
Jun-2023,"7,780.00",0.00,695.38,"1,216.36","9,691.75"
Jul-2023,"7,980.00",0.00,867.69,"1,560.00","10,407.69"
Aug-2023,"8,523.33",0.00,"1,052.31","1,928.18","11,503.82"
Sep-2023,"9,256.67",0.00,"1,236.92","2,337.27","12,830.86"
Oct-2023,"11,356.67",0.00,"1,421.54","2,746.36","15,524.57"


### **Calculate For Credit Loss For New Disbursements**

In [304]:
provision_for_credit_loss_for_all_new_disbursements = (
    expenses.calculate_provision_for_credit_loss_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df, parameters=parameters
    )
)
provision_for_credit_loss_for_all_new_disbursements

,sme_provision_for_credit_loss,b2b_provision_for_credit_loss,consumer_ssb_provision_for_credit_loss,consumer_pvt_provision_for_credit_loss,total
2023-01,"7,200.00",100.00,960.00,"1,800.00","10,060.00"
2023-02,"8,640.00",100.00,"1,200.00","2,250.00","12,190.00"
2023-03,"9,600.00",100.00,"1,440.00","3,240.00","14,380.00"
2023-04,"7,200.00",100.00,"1,600.00","3,600.00","12,500.00"
2023-05,"6,480.00",100.00,"1,920.00","4,320.00","12,820.00"
2023-06,"7,560.00",100.00,"1,920.00","4,860.00","14,440.00"
2023-07,"8,400.00",100.00,"2,240.00","5,670.00","16,410.00"
2023-08,"11,900.00",100.00,"2,400.00","6,075.00","20,475.00"
2023-09,"14,000.00",100.00,"2,400.00","6,750.00","23,250.00"
2023-10,"19,800.00",100.00,"2,400.00","6,750.00","29,050.00"


## **Existing Loans**

### **Generate Loan Schedules For Existing Loans**


In [305]:
existing_loans_schedules = interest_income.generate_loan_schedules_existing_loans(
    outstanding_balance=existing_loans["outstanding_balance"],
    interest_rate_monthly=existing_loans["interest_rate"],
    repayment_amount_monthly=existing_loans["repayment_amount"],
    valuation_date=VALUATION_DATE, 
    months_to_project=MONTHS_TO_FORECAST,
)
existing_loans_schedules.keys()

dict_keys(['interest', 'capital_repayment', 'outstanding_at_start'])

### **Capital Repayment for Existing Loans**

In [306]:
existing_loans_schedules['capital_repayment']

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
loan_number,,,,,,,,,,,,
40b0775c,25.29,25.40,25.51,25.63,25.74,25.86,25.98,26.10,26.21,26.33,26.45,0.00
f5c36555,48.67,49.32,49.98,50.65,51.33,52.01,52.71,53.41,54.13,54.85,55.59,0.00
75656bce,27.07,27.39,27.72,28.05,28.38,28.72,29.06,29.41,29.76,30.11,30.47,0.00
a7222f00,79.32,79.44,79.57,79.69,79.82,79.95,80.07,80.20,80.32,80.45,80.58,0.00
ad00cbf5,50.18,50.22,50.25,50.28,50.31,50.34,50.37,50.40,50.43,50.47,50.50,0.00
321ec75b,18.11,18.20,18.29,18.37,18.46,18.55,18.65,18.74,18.83,18.92,19.01,0.00
6747e1d4,20.32,20.56,20.81,21.07,21.33,21.59,21.85,22.11,22.38,22.66,22.93,0.00
5131c13c,14.44,14.55,14.66,14.77,14.89,15.00,15.12,15.24,15.35,15.47,15.59,0.00
a7222f00_1,14.84,14.93,15.02,15.11,15.21,15.30,15.39,15.49,15.58,15.68,15.78,0.00


In [307]:
capital_repayment_existing_loans = existing_loans_schedules['capital_repayment'].sum()
capital_repayment_existing_loans

Jan-2023   884.89
Feb-2023   891.70
Mar-2023   898.59
Apr-2023   905.55
May-2023   912.59
Jun-2023   919.70
Jul-2023   926.89
Aug-2023   934.16
Sep-2023   941.52
Oct-2023   948.95
Nov-2023   956.47
Dec-2023     0.00
dtype: float64

### **Intererest Income For Existing Loans**

In [308]:
interest_income_existing_loans = existing_loans_schedules['interest'].sum()
interest_income_existing_loans

Jan-2023   290.78
Feb-2023   283.97
Mar-2023   277.09
Apr-2023   270.13
May-2023   263.09
Jun-2023   255.98
Jul-2023   248.78
Aug-2023   241.51
Sep-2023   234.16
Oct-2023   226.73
Nov-2023   219.21
Dec-2023     0.00
dtype: float64

### **Calculate Other Income Existing Loans**

In [309]:
other_income_existing_loans_df = other_income.calculate_other_income_existing_loans(existing_loans=existing_loans, valuation_date=VALUATION_DATE, months_to_forecast=MONTHS_TO_FORECAST)
other_income_existing_loans_df

,credit_insurance_fee_existing_loans,admin_fee_existing_loans,total
Jan-2023,61.55,66.22,127.78
Feb-2023,61.55,66.22,127.78
Mar-2023,61.55,66.22,127.78
Apr-2023,61.55,66.22,127.78
May-2023,61.55,66.22,127.78
Jun-2023,61.55,66.22,127.78
Jul-2023,61.55,66.22,127.78
Aug-2023,61.55,66.22,127.78
Sep-2023,61.55,66.22,127.78
Oct-2023,61.55,66.22,127.78


## **Interest Income**

In [310]:
total_interest_income = interest_income.aggregate_new_and_existing_loans_interest_income(
    interest_income_new_disbursements_df=interest_income_new_disbursements_df,
    interest_income_existing_loans= interest_income_existing_loans,
    valuation_date= VALUATION_DATE, 
    months_to_forecast= MONTHS_TO_FORECAST
)
total_interest_income

Jan-2023       290.78
Feb-2023    90,123.97
Mar-2023   187,900.90
Apr-2023   289,132.99
May-2023   354,282.78
Jun-2023   405,013.17
Jul-2023   453,611.32
Aug-2023   499,318.37
Sep-2023   575,826.00
Oct-2023   668,308.56
Nov-2023   806,367.53
Dec-2023   950,524.03
dtype: float64

## **Other Income**

In [311]:
other_income_df = other_income.aggregate_other_income(
    admin_fee_for_all_new_disbursements_df=admin_fee_for_all_new_disbursements_df,
    admin_fee_existing_loans=other_income_existing_loans_df['admin_fee_existing_loans'],
    credit_insurance_fee_existing_loans=other_income_existing_loans_df['credit_insurance_fee_existing_loans'],
    credit_insurance_fee_for_all_new_disbursements_df=credit_insurance_fee_for_all_new_disbursements, 
    valuation_date=VALUATION_DATE, 
    months_to_forecast=MONTHS_TO_FORECAST
)
other_income_df

,admin_fee,credit_insurance_fee,total
Jan-2023,"7,000.91","2,827.43","9,828.34"
Feb-2023,"15,364.26","6,164.77","21,529.03"
Mar-2023,"24,919.93","9,979.04","34,898.96"
Apr-2023,"32,646.22","13,061.55","45,707.78"
May-2023,"40,113.77","16,040.57","56,154.35"
Jun-2023,"48,644.96","19,445.05","68,090.01"
Jul-2023,"52,244.68","20,876.94","73,121.62"
Aug-2023,"57,745.34","23,069.20","80,814.54"
Sep-2023,"64,400.53","25,723.28","90,123.81"
Oct-2023,"77,889.07","31,110.69","108,999.76"


## **Depreciation**

In [312]:
depreciations_and_nbvs = depreciation.calculate_depreciations_and_nbvs(
    details_of_existing_assets=details_of_existing_assets,
    details_of_new_assets=details_of_new_assets,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST
)
depreciations_and_nbvs.keys()

dict_keys(['nbvs', 'dpns'])

In [313]:
depreciations_and_nbvs['dpns']

,Building,Buildings,Vehicle,Vehicles,total
Jan-2023,"5,000.00","5,000.00","2,500.00","2,000.00","14,500.00"
Feb-2023,"4,500.00","4,500.00","1,875.00","1,600.00","12,475.00"
Mar-2023,"4,050.00","4,050.00","1,406.25","1,280.00","10,786.25"
Apr-2023,"3,645.00","3,645.00","1,054.69","1,024.00","9,368.69"
May-2023,"3,280.50","3,280.50",791.02,819.20,"8,171.22"
Jun-2023,"2,952.45","2,952.45",0.00,0.00,"5,904.90"
Jul-2023,"2,657.20","2,657.20",0.00,0.00,"5,314.41"
Aug-2023,"2,391.48","2,391.48",0.00,0.00,"4,782.97"
Sep-2023,"2,152.34","2,152.34",0.00,0.00,"4,304.67"
Oct-2023,"1,937.10","1,937.10",0.00,0.00,"3,874.20"


In [314]:
depreciations_and_nbvs['dpns']

,Building,Buildings,Vehicle,Vehicles,total
Jan-2023,"5,000.00","5,000.00","2,500.00","2,000.00","14,500.00"
Feb-2023,"4,500.00","4,500.00","1,875.00","1,600.00","12,475.00"
Mar-2023,"4,050.00","4,050.00","1,406.25","1,280.00","10,786.25"
Apr-2023,"3,645.00","3,645.00","1,054.69","1,024.00","9,368.69"
May-2023,"3,280.50","3,280.50",791.02,819.20,"8,171.22"
Jun-2023,"2,952.45","2,952.45",0.00,0.00,"5,904.90"
Jul-2023,"2,657.20","2,657.20",0.00,0.00,"5,314.41"
Aug-2023,"2,391.48","2,391.48",0.00,0.00,"4,782.97"
Sep-2023,"2,152.34","2,152.34",0.00,0.00,"4,304.67"
Oct-2023,"1,937.10","1,937.10",0.00,0.00,"3,874.20"


## **Salaries**

In [315]:
salaries_and_pension_and_statutory_contributions_df  = expenses.calculate_salaries_and_pension_and_statutory_contributions(
    new_disbursements_df=new_disbursements_df,
    parameters=parameters,
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,
)
salaries_and_pension_and_statutory_contributions_df

,agent_salaries,credit_officer_salaries,other_staff_salaries,total,pensions_and_statutory_contributions
Jan-2023,"6,000.00","4,200.00","17,000.00","27,200.00","3,089.92"
Feb-2023,"7,500.00","4,920.00","17,000.00","29,420.00","3,342.11"
Mar-2023,"9,900.00","5,400.00","17,000.00","32,300.00","3,669.28"
Apr-2023,"11,000.00","4,200.00","17,000.00","32,200.00","3,657.92"
May-2023,"13,200.00","4,140.00","17,000.00","34,340.00","3,901.02"
Jun-2023,"14,100.00","4,680.00","17,000.00","35,780.00","4,064.61"
Jul-2023,"16,450.00","5,400.00","17,000.00","38,850.00","4,413.36"
Aug-2023,"17,625.00","7,350.00","17,000.00","41,975.00","4,768.36"
Sep-2023,"18,750.00","8,400.00","17,000.00","44,150.00","5,015.44"
Oct-2023,"18,750.00","11,700.00","17,000.00","47,450.00","5,390.32"


## **Provisions**


In [316]:
trade_receivables_schedule = expenses.generate_trade_receivables_schedule(
    opening_trade_receivables=opening_balances['TRADE_RECEIVABLES'].iat[0], 
    receipts_from_trade_receivables=parameters.loc["RECEIPTS_FROM_TRADE_RECEIVABLES"], 
    new_trade_receivables=parameters.loc['NEW_TRADE_RECEIVABLES'], 
    months_to_forecast=MONTHS_TO_FORECAST, 
    valuation_date=VALUATION_DATE
)
trade_receivables_schedule

,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
Opening Balance,1000000,999500,999000,998500,998000,997500,997000,996500,996000,995500,995000,994500
New Trade Receivables,500,500,500,500,500,500,500,500,500,500,500,500
Receipts From Trade Receivables,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000
Closing Balance,999500,999000,998500,998000,997500,997000,996500,996000,995500,995000,994500,994000


In [317]:
provision_for_credit_loss_for_all_new_disbursements = (
    expenses.calculate_provision_for_credit_loss_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df, parameters=parameters
    )
)
provision_for_credit_loss_for_all_new_disbursements

,sme_provision_for_credit_loss,b2b_provision_for_credit_loss,consumer_ssb_provision_for_credit_loss,consumer_pvt_provision_for_credit_loss,total
2023-01,"7,200.00",100.00,960.00,"1,800.00","10,060.00"
2023-02,"8,640.00",100.00,"1,200.00","2,250.00","12,190.00"
2023-03,"9,600.00",100.00,"1,440.00","3,240.00","14,380.00"
2023-04,"7,200.00",100.00,"1,600.00","3,600.00","12,500.00"
2023-05,"6,480.00",100.00,"1,920.00","4,320.00","12,820.00"
2023-06,"7,560.00",100.00,"1,920.00","4,860.00","14,440.00"
2023-07,"8,400.00",100.00,"2,240.00","5,670.00","16,410.00"
2023-08,"11,900.00",100.00,"2,400.00","6,075.00","20,475.00"
2023-09,"14,000.00",100.00,"2,400.00","6,750.00","23,250.00"
2023-10,"19,800.00",100.00,"2,400.00","6,750.00","29,050.00"


In [318]:
provisions = expenses.calculate_provisions(
    trade_receivables_schedule=trade_receivables_schedule,
    provision_for_credit_loss_for_all_new_disbursements_df=provision_for_credit_loss_for_all_new_disbursements,
    parameters=parameters, 
    valuation_date=VALUATION_DATE, 
    months_to_forecast=MONTHS_TO_FORECAST
)

provisions 

,provision_for_bad_debts,provision_for_credit_loss,total
Jan-2023,99.95,"20,120.00","20,219.95"
Feb-2023,99.90,"24,380.00","24,479.90"
Mar-2023,99.85,"28,760.00","28,859.85"
Apr-2023,99.80,"25,000.00","25,099.80"
May-2023,99.75,"25,640.00","25,739.75"
Jun-2023,99.70,"28,880.00","28,979.70"
Jul-2023,99.65,"32,820.00","32,919.65"
Aug-2023,99.60,"40,950.00","41,049.60"
Sep-2023,99.55,"46,500.00","46,599.55"
Oct-2023,99.50,"58,100.00","58,199.50"


### **Calculating Finance Costs**

In [319]:
finance_costs = expenses.calculate_finance_costs(
    details_of_existing_long_term_borrowing = details_of_existing_long_term_borrowing,
    details_of_existing_short_term_borrowing = details_of_existing_short_term_borrowing,
    details_of_new_short_term_borrowing = details_of_new_short_term_borrowing,
    details_of_new_long_term_borrowing =details_of_new_long_term_borrowing,
    valuation_date= VALUATION_DATE,
    months_to_forecast= MONTHS_TO_FORECAST,
)

finance_costs.loc['total']

Jan-2023   67,811.10
Feb-2023   67,811.10
Mar-2023   67,811.10
Apr-2023   67,486.10
May-2023   67,486.10
Jun-2023   65,680.36
Jul-2023   65,680.36
Aug-2023   64,430.36
Sep-2023   54,013.69
Oct-2023   54,013.69
Nov-2023   53,430.36
Dec-2023   45,180.36
Name: total, dtype: float64

## **Income Statement**

In [320]:
income_statement_index = pd.read_csv("./data_templates/income_statement_index.csv")
income_statement_index

,INCOME_STATEMENT
0,Interest Income
1,Other Income
2,Total Revenue
3,MANAGEMENT EXPENSES
4,STAFF COSTS
5,Salaries
6,Pensions & Statutory Contributions
7,Training
8,Bonus Provision
9,Retrenchments


In [321]:
importlib.reload(income_statement)

<module 'application.modeling.income_statement' from 'c:\\Users\\HP\\Desktop\\Claxon\\Budgeting\\app\\modeling\\income_statement.py'>

In [322]:
income_statement_df = income_statement.generate_income_statement_template(
    income_statement_index=income_statement_index,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Revenue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bonus Provision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Calculate Variable Expenses**

In [245]:
variable_expenses = expenses.calculate_variable_expenses(
    variable_inputs_income_statement=variable_inputs_income_statement,
    parameters=parameters,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)
variable_expenses

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Travel Costs,"7,080.16","7,221.76","7,366.20","7,513.52","7,663.79","7,817.07","7,973.41","8,132.88","8,295.54","8,461.45","8,630.68","8,803.29"
Entertainment,"7,760.67","7,915.88","8,074.20","8,235.69","8,400.40","8,568.41","8,739.77","8,914.57","9,092.86","9,274.72","9,460.21","9,649.42"
Marketing Costs,"7,796.37","7,952.30","8,111.34","8,273.57","8,439.04","8,607.82","8,779.98","8,955.58","9,134.69","9,317.38","9,503.73","9,693.81"
Group Marketing Costs,"7,659.18","7,812.36","7,968.61","8,127.98","8,290.54","8,456.35","8,625.48","8,797.99","8,973.95","9,153.43","9,336.50","9,523.23"
Rental Costs,"7,749.79","7,904.79","8,062.88","8,224.14","8,388.62","8,556.39","8,727.52","8,902.07","9,080.11","9,261.72","9,446.95","9,635.89"
Subscriptions,"8,444.75","8,613.64","8,785.92","8,961.64","9,140.87","9,323.69","9,510.16","9,700.36","9,894.37","10,092.26","10,294.10","10,499.99"
Insurance,"7,590.16","7,741.96","7,896.80","8,054.74","8,215.83","8,380.15","8,547.75","8,718.71","8,893.08","9,070.94","9,252.36","9,437.41"
Repairs And Maintenance,"8,262.68","8,427.93","8,596.49","8,768.42","8,943.79","9,122.67","9,305.12","9,491.22","9,681.05","9,874.67","10,072.16","10,273.60"
Utilities,"7,537.97","7,688.73","7,842.50","7,999.35","8,159.34","8,322.53","8,488.98","8,658.76","8,831.93","9,008.57","9,188.74","9,372.52"


### **Calculate Change In Provision For Credit Loss**

In [246]:
change_in_provisin_for_credit_loss = (
    expenses.calculate_change_in_provision_for_credit_loss(
        provision_for_credit_loss=provision_for_credit_loss_for_all_new_disbursements['total'],
        provision_for_credit_loss_opening_balances=float(
            opening_balances["PROVISION_FOR_CREDIT_LOSS"].iat[0]
        ),
        valuation_date=VALUATION_DATE, 
        months_to_forecast=MONTHS_TO_FORECAST
    )
)
change_in_provisin_for_credit_loss

Jan-2023       60.00
Feb-2023    2,130.00
Mar-2023    2,190.00
Apr-2023   -1,880.00
May-2023      320.00
Jun-2023    1,620.00
Jul-2023    1,970.00
Aug-2023    4,065.00
Sep-2023    2,775.00
Oct-2023    5,800.00
Nov-2023    2,700.00
Dec-2023        0.00
Name: total, dtype: float64

### **Calculate Business Acquisition**

In [247]:
new_disbursements_df.columns

Index(['b2b_disbursements', 'sme_disbursements', 'consumer_ssb_disbursements',
       'consumer_pvt_disbursements', 'total'],
      dtype='object')

In [248]:
business_acquisition = expenses.calculate_business_acqusition(
    business_acquisition_percent=parameters.loc['BUSINESS_ACQUISITION_PERCENT'],
    agent_contribution_percent=parameters.loc['AGENT_CONTRIBUTION_PERCENT'],
    consumer_ssb_disbursements=new_disbursements_df['consumer_ssb_disbursements'],
    consumer_pvt_disbursements=new_disbursements_df['consumer_pvt_disbursements'],
)
business_acquisition

Jan-2023   1,620.00
Feb-2023   2,025.00
Mar-2023   2,484.00
Apr-2023   2,760.00
May-2023   3,312.00
Jun-2023   3,366.00
Jul-2023   3,927.00
Aug-2023   4,207.50
Sep-2023   4,275.00
Oct-2023   4,275.00
Nov-2023   4,275.00
Dec-2023   4,275.00
dtype: float64

### **Insert Revenue**

In [249]:
income_statement_df = income_statement.insert_revenue(
    income_statement=income_statement_df,
    interest_income=total_interest_income,
    other_income=other_income_df['total'],
)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,290.78,"90,123.97","187,900.90","289,132.99","354,282.78","405,013.17","453,611.32","499,318.37","575,826.00","668,308.56","806,367.53","950,524.03"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Total Revenue,"10,119.12","111,653.01","222,799.87","334,840.76","410,437.13","473,103.18","526,732.94","580,132.90","665,949.82","777,308.31","938,213.23","1,103,192.04"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bonus Provision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Insert Static And Variable Inputs**

In [250]:
income_statement_df = income_statement.insert_static_and_variable_inputs(
    income_statement=income_statement_df,
    static_inputs_income_statement=static_inputs_income_statement, variable_expenses=variable_expenses
)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,290.78,"90,123.97","187,900.90","289,132.99","354,282.78","405,013.17","453,611.32","499,318.37","575,826.00","668,308.56","806,367.53","950,524.03"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Total Revenue,"10,119.12","111,653.01","222,799.87","334,840.76","410,437.13","473,103.18","526,732.94","580,132.90","665,949.82","777,308.31","938,213.23","1,103,192.04"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


### **Insert Salaries and Pensions And Statutory Contributions**

In [251]:
income_statement_df = income_statement.insert_salaries_and_pensions_and_statutory_contributions(
    income_statement=income_statement_df,
    salaries_and_pension_and_statutory_contributions_df = salaries_and_pension_and_statutory_contributions_df,
)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,290.78,"90,123.97","187,900.90","289,132.99","354,282.78","405,013.17","453,611.32","499,318.37","575,826.00","668,308.56","806,367.53","950,524.03"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Total Revenue,"10,119.12","111,653.01","222,799.87","334,840.76","410,437.13","473,103.18","526,732.94","580,132.90","665,949.82","777,308.31","938,213.23","1,103,192.04"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


### **Insert Depreciation**

In [252]:
income_statement_df = income_statement.insert_depreciation(
    income_statement=income_statement_df, depreciation=depreciations_and_nbvs['dpns']['total'])
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,290.78,"90,123.97","187,900.90","289,132.99","354,282.78","405,013.17","453,611.32","499,318.37","575,826.00","668,308.56","806,367.53","950,524.03"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Total Revenue,"10,119.12","111,653.01","222,799.87","334,840.76","410,437.13","473,103.18","526,732.94","580,132.90","665,949.82","777,308.31","938,213.23","1,103,192.04"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


### **Insert Bad Debts Provision and Change in Provision For Credit Loss**

In [253]:
income_statement_df = income_statement.insert_bad_debts_provision(
    income_statement=income_statement_df, 
    provision_for_bad_debts=provisions['provision_for_bad_debts'],
    change_in_provisin_for_credit_loss=change_in_provisin_for_credit_loss,
)

income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,290.78,"90,123.97","187,900.90","289,132.99","354,282.78","405,013.17","453,611.32","499,318.37","575,826.00","668,308.56","806,367.53","950,524.03"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Total Revenue,"10,119.12","111,653.01","222,799.87","334,840.76","410,437.13","473,103.18","526,732.94","580,132.90","665,949.82","777,308.31","938,213.23","1,103,192.04"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


In [254]:
provision_for_credit_loss  = provision_for_credit_loss_for_all_new_disbursements['total']
provision_for_credit_loss

2023-01   10,060.00
2023-02   12,190.00
2023-03   14,380.00
2023-04   12,500.00
2023-05   12,820.00
2023-06   14,440.00
2023-07   16,410.00
2023-08   20,475.00
2023-09   23,250.00
2023-10   29,050.00
2023-11   31,750.00
2023-12   31,750.00
Freq: M, Name: total, dtype: float64

### **Insert Business Acquisition**

In [255]:
income_statement_df = income_statement.insert_business_acquisition(
    income_statement=income_statement_df, business_acquisition=business_acquisition)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,290.78,"90,123.97","187,900.90","289,132.99","354,282.78","405,013.17","453,611.32","499,318.37","575,826.00","668,308.56","806,367.53","950,524.03"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Total Revenue,"10,119.12","111,653.01","222,799.87","334,840.76","410,437.13","473,103.18","526,732.94","580,132.90","665,949.82","777,308.31","938,213.23","1,103,192.04"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


### **Aggregations**

In [256]:
income_statement_df = income_statement.aggregate_staff_costs(income_statement_df)
income_statement_df = income_statement.aggregate_travel_and_entertainment(income_statement_df)
income_statement_df = income_statement.aggregate_marketing_and_public_relations(income_statement_df)
income_statement_df = income_statement.aggregate_office_costs(income_statement_df)
income_statement_df = income_statement.aggregate_professional_fees(income_statement_df)
income_statement_df = income_statement.aggregate_communication_costs(income_statement_df)
income_statement_df = income_statement.aggregate_motor_vehicle_costs(income_statement_df)
income_statement_df = income_statement.aggregate_other_costs(income_statement_df)
income_statement_df = income_statement.aggregate_investment_income(income_statement_df)
income_statement_df = income_statement.aggregate_finance_costs(income_statement_df)

### **Calculating Total Expenses and EBIDTA**

In [257]:
income_statement_df = income_statement.calculate_total_expenses(income_statement_df)
income_statement_df = income_statement.calculate_ebidta(income_statement_df)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,290.78,"90,123.97","187,900.90","289,132.99","354,282.78","405,013.17","453,611.32","499,318.37","575,826.00","668,308.56","806,367.53","950,524.03"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Total Revenue,"10,119.12","111,653.01","222,799.87","334,840.76","410,437.13","473,103.18","526,732.94","580,132.90","665,949.82","777,308.31","938,213.23","1,103,192.04"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


### **Insert Finance Cost**

In [258]:
income_statement_df.loc['Finance Costs'] = finance_costs.loc['total']

### **Aggregate Finance Cost**

In [259]:
income_statement_df = income_statement.aggregate_finance_costs(income_statement_df)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,290.78,"90,123.97","187,900.90","289,132.99","354,282.78","405,013.17","453,611.32","499,318.37","575,826.00","668,308.56","806,367.53","950,524.03"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Total Revenue,"10,119.12","111,653.01","222,799.87","334,840.76","410,437.13","473,103.18","526,732.94","580,132.90","665,949.82","777,308.31","938,213.23","1,103,192.04"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


### **Calculate Profit Before Tax**

In [260]:
income_statement_df = income_statement.calculate_profit_before_tax(income_statement_df)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,290.78,"90,123.97","187,900.90","289,132.99","354,282.78","405,013.17","453,611.32","499,318.37","575,826.00","668,308.56","806,367.53","950,524.03"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Total Revenue,"10,119.12","111,653.01","222,799.87","334,840.76","410,437.13","473,103.18","526,732.94","580,132.90","665,949.82","777,308.31","938,213.23","1,103,192.04"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


### **Calculate Tax**

In [261]:
income_statement_df = income_statement.calculate_tax(income_statement=income_statement_df,tax_rate=parameters.loc['TAX_RATE'])
income_statement_df

c:\Users\HP\anaconda3\Lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: invalid value encountered in maximum
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,290.78,"90,123.97","187,900.90","289,132.99","354,282.78","405,013.17","453,611.32","499,318.37","575,826.00","668,308.56","806,367.53","950,524.03"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Total Revenue,"10,119.12","111,653.01","222,799.87","334,840.76","410,437.13","473,103.18","526,732.94","580,132.90","665,949.82","777,308.31","938,213.23","1,103,192.04"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


In [262]:
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,290.78,"90,123.97","187,900.90","289,132.99","354,282.78","405,013.17","453,611.32","499,318.37","575,826.00","668,308.56","806,367.53","950,524.03"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Total Revenue,"10,119.12","111,653.01","222,799.87","334,840.76","410,437.13","473,103.18","526,732.94","580,132.90","665,949.82","777,308.31","938,213.23","1,103,192.04"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


## **Direct Cashflow**

In [263]:
direct_cashflow_df = direct_cashflow.generate_direct_cashflow_template(
    valuation_date=VALUATION_DATE, months_to_forecast=MONTHS_TO_FORECAST
)

direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,,,,,,,,,,,,
Receipts From Trade Receivables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Capital Repayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Issue Of Shares,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Cash Inflows,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CASH OUTFLOWS,,,,,,,,,,,,


### **Insert Available Direct Cashflow Items**

#### **From Parameters**

In [264]:
direct_cashflow_df.loc['Receipts From Trade Receivables'] = parameters.loc['RECEIPTS_FROM_TRADE_RECEIVABLES']
direct_cashflow_df.loc['Issue Of Shares'] = parameters.loc['ISSUE_OF_SHARES']
direct_cashflow_df.loc['Tax Paid'] = parameters.loc['TAX_PAID']
direct_cashflow_df.loc['Dividend Paid'] = parameters.loc['DIVIDEND_PAID']

#### **From Calculations/Income Statement**

In [265]:
direct_cashflow_df.loc['Interest Income'] = income_statement_df.loc['Interest Income']
direct_cashflow_df.loc['Other Income'] = income_statement_df.loc['Other Income']
direct_cashflow_df.loc['Interest Expense'] = income_statement_df.loc['Finance Costs']
direct_cashflow_df.loc['Disbursements'] = new_disbursements_df["total"].reindex(
    pd.PeriodIndex(new_disbursements_df["total"].index, freq="M").strftime("%b-%Y")
)

In [266]:
direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,,,,,,,,,,,,
Receipts From Trade Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Capital Repayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,290.78,"90,123.97","187,900.90","289,132.99","354,282.78","405,013.17","453,611.32","499,318.37","575,826.00","668,308.56","806,367.53","950,524.03"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Issue Of Shares,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00
Short Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Cash Inflows,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CASH OUTFLOWS,,,,,,,,,,,,


### **Calculate Operating Expenses**

In [267]:
operating_expenses = direct_cashflow.calculate_operating_expenses(income_statement=income_statement_df)
operating_expenses

Jan-2023   299,404.18
Feb-2023   299,078.04
Mar-2023   304,602.20
Apr-2023   325,184.80
May-2023   321,003.48
Jun-2023   332,025.96
Jul-2023   336,694.64
Aug-2023   346,326.86
Sep-2023   359,085.76
Oct-2023   355,671.39
Nov-2023   374,455.79
Dec-2023   361,106.03
dtype: object

### **Insert Operating Expenses**

In [268]:
direct_cashflow_df.loc['Operating Expenses']  = operating_expenses
direct_cashflow_df


,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,,,,,,,,,,,,
Receipts From Trade Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Capital Repayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,290.78,"90,123.97","187,900.90","289,132.99","354,282.78","405,013.17","453,611.32","499,318.37","575,826.00","668,308.56","806,367.53","950,524.03"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Issue Of Shares,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00
Short Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Cash Inflows,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CASH OUTFLOWS,,,,,,,,,,,,


### **Calculate Capital Expenses**

In [269]:
capital_expenses = direct_cashflow.calculate_capital_expenses(
    details_of_new_assets=details_of_new_assets,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)
capital_expenses

Jan-2023        0
Feb-2023        0
Mar-2023        0
Apr-2023        0
May-2023        0
Jun-2023    10000
Jul-2023        0
Aug-2023        0
Sep-2023    50000
Oct-2023        0
Nov-2023        0
Dec-2023        0
Name: cost, dtype: int64

### **Insert Capital Expenses**

In [270]:
direct_cashflow_df.loc['Capital Expenses'] = capital_expenses

### **Calculate Direct Cashflow Borrowing**

In [271]:
short_term_borrowing = direct_cashflow.calculate_direct_cashflow_borrowing(
    details_of_new_borrowing=details_of_new_short_term_borrowing,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)
short_term_borrowing

Jan-2023         0
Feb-2023         0
Mar-2023         0
Apr-2023         0
May-2023         0
Jun-2023         0
Jul-2023    600000
Aug-2023         0
Sep-2023    500000
Oct-2023    500000
Nov-2023         0
Dec-2023         0
Name: principal, dtype: int64

In [272]:
long_term_borrowing = direct_cashflow.calculate_direct_cashflow_borrowing(
   details_of_new_borrowing= details_of_new_long_term_borrowing, 
   valuation_date=VALUATION_DATE, 
   months_to_forecast=MONTHS_TO_FORECAST
)
long_term_borrowing

Jan-2023          0
Feb-2023          0
Mar-2023          0
Apr-2023          0
May-2023          0
Jun-2023          0
Jul-2023          0
Aug-2023     200000
Sep-2023     400000
Oct-2023    1000000
Nov-2023          0
Dec-2023          0
Name: principal, dtype: int64

### **Insert Direct Cashflow Borrowing**

In [273]:
direct_cashflow_df.loc['Short Term Borrowing'] = short_term_borrowing
direct_cashflow_df.loc['Long Term Borrowing'] = long_term_borrowing

### **Calculate Capital Repayments**

In [274]:
capital_repayment = helper.add_series(
[existing_loans_schedules['capital_repayment'].sum()
    , capital_repayment_new_disbursements_df['total']]
)
capital_repayment

Jan-2023       884.89
Feb-2023    57,064.61
Mar-2023   130,066.30
Apr-2023   219,706.55
May-2023   298,143.94
Jun-2023   381,231.98
Jul-2023   480,117.56
Aug-2023   514,505.17
Sep-2023   557,928.09
Oct-2023   609,402.17
Nov-2023   711,656.07
Dec-2023   842,349.66
Jan-2024   966,750.42
dtype: float64

### **Insert Capital Repayment**

In [275]:
direct_cashflow_df.loc['Capital Repayment'] = capital_repayment

### **Calculate Capital Repayments on Borrowings**


In [276]:
capital_repayment_on_borrowings_df = direct_cashflow.calculate_capital_repayment_on_borrowings(
        details_of_existing_long_term_borrowing=details_of_existing_long_term_borrowing,
        details_of_existing_short_term_borrowing=details_of_existing_short_term_borrowing,
        details_of_new_short_term_borrowing=details_of_new_short_term_borrowing,
        details_of_new_long_term_borrowing=details_of_new_long_term_borrowing,
        valuation_date=VALUATION_DATE,
        months_to_forecast=MONTHS_TO_FORECAST,
    )

capital_repayment_on_borrowings_df

,capital_repayment_on_new_borrowings,capital_repayment_on_existing_borrowings,total
Jan-2023,0,0,0
Feb-2023,0,0,0
Mar-2023,0,0,0
Apr-2023,0,30000,30000
May-2023,0,0,0
Jun-2023,0,120383,120383
Jul-2023,0,0,0
Aug-2023,0,125000,125000
Sep-2023,0,500000,500000
Oct-2023,0,0,0


### **Insert Capital Repayment on New Borrowings**

In [277]:
direct_cashflow_df.loc[
    "Capital Repayment On Borrowings"
] = capital_repayment_on_borrowings_df['total']

### **Calculate Totals**

In [278]:
direct_cashflow_df.loc["Total Cash Inflows"] = direct_cashflow_df.loc[
    "Receipts From Trade Receivables":"Long Term Borrowing"
].sum()

direct_cashflow_df.loc["Total Cash Outflows"] = direct_cashflow_df.loc[
    "Disbursements":"Tax Paid"
].sum()

direct_cashflow_df.loc['Net Increase/Decrease In Cash'] = direct_cashflow_df.loc['Total Cash Inflows'] - direct_cashflow_df.loc['Total Cash Outflows']

### **Calculate Opening and Closing Balances**

In [279]:
direct_cashflow_df= direct_cashflow.calculate_opening_and_closing_balances_for_direct_cashflows(
direct_cashflow=direct_cashflow_df, 
cash_on_hand_opening_balance= opening_balances['CASH_ON_HAND'].iat[0]
)
direct_cashflow_df

,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
CASH INFLOWS,,,,,,,,,,,,
Receipts From Trade Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Capital Repayment,884.89,"57,064.61","130,066.30","219,706.55","298,143.94","381,231.98","480,117.56","514,505.17","557,928.09","609,402.17","711,656.07","842,349.66"
Interest Income,290.78,"90,123.97","187,900.90","289,132.99","354,282.78","405,013.17","453,611.32","499,318.37","575,826.00","668,308.56","806,367.53","950,524.03"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Issue Of Shares,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00
Short Term Borrowing,0,0,0,0,0,0,600000,0,500000,500000,0,0
Long Term Borrowing,0,0,0,0,0,0,0,200000,400000,1000000,0,0
Total Cash Inflows,"12,004.01","169,717.62","353,866.16","555,547.31","709,581.07","855,335.16","1,607,850.50","1,295,638.08","2,124,877.91","2,887,810.49","1,650,869.30","1,946,541.70"
CASH OUTFLOWS,,,,,,,,,,,,


### **Calculate IMTT (2% Tax)**

In [280]:
income_statement_df.loc['2% Taxation']  = direct_cashflow_df.loc['Total Cash Outflows']*IMTT

## **Calculate Profit and Loss for the Period**


In [281]:
income_statement_df = income_statement.calculate_profit_or_loss_for_period(income_statement_df)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,290.78,"90,123.97","187,900.90","289,132.99","354,282.78","405,013.17","453,611.32","499,318.37","575,826.00","668,308.56","806,367.53","950,524.03"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Total Revenue,"10,119.12","111,653.01","222,799.87","334,840.76","410,437.13","473,103.18","526,732.94","580,132.90","665,949.82","777,308.31","938,213.23","1,103,192.04"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


## **Loan Book**

In [282]:
loan_book_df = direct_cashflow.generate_loan_book_template(
    valuation_date=VALUATION_DATE, months_to_forecast=MONTHS_TO_FORECAST
)
loan_book_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
New Disbursements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Capital Repayments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Closing Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Insert Loan Book Items**

In [283]:
loan_book_df = direct_cashflow.insert_loan_book_items(
    loan_book=loan_book_df, 
    opening_balance_on_loan_book=float(opening_balances['LOAN_BOOK'].iat[0]), 
    capital_repayment=capital_repayment, 
    disbursements=new_disbursements_df['total'].reindex(new_disbursements_df['total'].index.strftime('%b-%Y'))
)
loan_book_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,"2,800,000.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
New Disbursements,"490,000.00","592,000.00","706,000.00","610,000.00","622,000.00","712,000.00","808,000.00","1,010,000.00","1,160,000.00","1,450,000.00","1,585,000.00","1,585,000.00"
Capital Repayments,884.89,"57,064.61","130,066.30","219,706.55","298,143.94","381,231.98","480,117.56","514,505.17","557,928.09","609,402.17","711,656.07","842,349.66"
Closing Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Calculate Opening and Closing Balances for Loan Book**

In [323]:
loan_book_df = direct_cashflow.calculate_opening_and_closing_balances_for_loan_book(
    loan_book_df
)

loan_book_df

,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
Opening Balance,"2,800,000.00","3,290,884.89","3,939,949.50","4,776,015.80","5,605,722.35","6,525,866.29","7,619,098.27","8,907,215.83","10,431,721.00","12,149,649.10","14,209,051.27","16,505,707.34"
New Disbursements,"490,000.00","592,000.00","706,000.00","610,000.00","622,000.00","712,000.00","808,000.00","1,010,000.00","1,160,000.00","1,450,000.00","1,585,000.00","1,585,000.00"
Capital Repayments,884.89,"57,064.61","130,066.30","219,706.55","298,143.94","381,231.98","480,117.56","514,505.17","557,928.09","609,402.17","711,656.07","842,349.66"
Closing Balance,"3,290,884.89","3,939,949.50","4,776,015.80","5,605,722.35","6,525,866.29","7,619,098.27","8,907,215.83","10,431,721.00","12,149,649.10","14,209,051.27","16,505,707.34","18,933,057.01"


## **Balance Sheet**


In [326]:
from enum import Enum

class FinalFiles(str, Enum):
    income_statement_df = "income_statement_df"
    direct_cashflow_df = "direct_cashflow_df"
    loan_book_df = "loan_book_df"

# Convert the enum to a dictionary
final_files_dict = [item.value for item in FinalFiles]

print(final_files_dict)


['income_statement_df', 'direct_cashflow_df', 'loan_book_df']


In [328]:
get_list_from_string_enum(FinalFiles)

['income_statement_df', 'direct_cashflow_df', 'loan_book_df']